# Layers and Modules

A **module is a set of layers**. As such, it could be a single layer, a generic subset of the model or the entire model.\
Each module must have a forward and a backpropagation method. It may, or may not, have parameters inside itself.\
Since auto-differentiation provides us with backpropagation, we are only intersted about parameters and forward propagation.

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

Let's revisit the code for implementing MLPs.

In [3]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

X = torch.rand(2, 20)
net(X).shape

torch.Size([2, 10])

# A custom module

Let's create a module with a hidden layer of 256 hidden units, and a 10-dimensional output layer. We modify the parent's class only in order to provide the essential features that a layer must give.

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
net(X).shape

/home/pachy/.cache/pypoetry/virtualenvs/dive_into_deep_learning-88_68WVw-py3.10/lib/python3.10/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 10])